## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
nums = list(range(1,1001))
len(nums)

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(nums, numSlices=10)
print(type(rdd))

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
rdd.count()

1000

In [6]:
rdd.first()

1

In [7]:
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [19]:
## Note: When you are dealing with big data, this could make your computer crash! It's best to avoid using the collect() method
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [9]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([7.23893873e-01, 3.67195435e-01, 1.43253501e+00, 2.90081638e+00,
       3.89022842e+00, 4.33606903e+00, 6.32383947e+00, 6.70035664e+00,
       3.20103823e+00, 5.79081707e+00, 2.97244512e+00, 2.93095470e+00,
       3.98547989e+00, 6.06452495e+00, 4.35004079e+00, 4.92443107e+00,
       5.95923930e+00, 2.50750397e+00, 2.27527578e+00, 7.02445861e+00,
       7.02422862e+00, 2.62677659e+00, 2.28322169e+01, 8.90080514e+00,
       1.47838466e+01, 4.23945916e+00, 1.25360285e+01, 1.84145504e+01,
       1.98265717e+01, 2.52218168e+01, 1.73501762e+01, 2.46813266e+01,
       1.79188235e+01, 3.24826870e+01, 2.66799249e+00, 2.95487476e+01,
       2.97007856e+01, 5.10217463e+00, 3.56648562e+01, 2.51306693e+01,
       5.31927441e+00, 1.25511712e+01, 4.10244075e+00, 3.68263984e+01,
       3.01553699e+01, 1.31074400e+01, 5.88248518e+00, 4.64688134e+01,
       4.67020887e+01, 2.16699755e+01, 4.76030499e+01, 4.67793691e+01,
       3.22231321e+01, 1.32267140e+01, 3.22272678e+01, 4.41545441e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [10]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.7238938726456575, 0.3671954346482518, 1.432535011508377, 2.9008163775145563]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [11]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [12]:
revenue_minus_tax.take(10)

[0.6659823628340049,
 0.33781979987639166,
 1.317932210587707,
 2.668751067313392,
 3.579010148734112,
 3.9891835110123277,
 5.8179323115221235,
 6.164328108941171,
 2.944955167919232,
 5.327551707832705]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [16]:
discounted.take(10)

[0.5993841265506045,
 0.3040378198887525,
 1.1861389895289363,
 2.401875960582053,
 3.221109133860701,
 3.590265159911095,
 5.236139080369911,
 5.547895298047054,
 2.650459651127309,
 4.794796537049435]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[789.2085757956369,
 778.4481807786963,
 747.7713792570837,
 736.5694906994293,
 731.1143416524235,
 727.2087048751125,
 725.9393164200671,
 723.7369658750912,
 722.190737084896,
 722.1709450982495,
 709.3995408199478,
 708.3217630494637,
 700.7575824418129,
 690.311216982604,
 679.8741536632564]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [101]:
discounted.toDebugString()

b'(10) PythonRDD[31] at RDD at PythonRDD.scala:49 []\n |   ParallelCollectionRDD[23] at parallelize at PythonRDD.scala:184 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [102]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.7461417892908068, 0.6178054015327881), (1.0516980354349175, 0.8708059733401117), (2.017327151511592, 1.6703468814515983), (0.17555392107371448, 0.1453586466490356), (4.3624584377810995, 3.61211558648275), (3.14786872854041, 2.6064353072314597), (5.75731245351263, 4.767054711508457), (5.075186267246741, 4.202254229280302), (3.9035082665726355, 3.232104844722142), (5.738197298803666, 4.751227363409436)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [104]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.7461417892908068, 0.6178054015327881, 1.0516980354349175, 0.8708059733401117, 2.017327151511592, 1.6703468814515983, 0.17555392107371448, 0.1453586466490356, 4.3624584377810995, 3.61211558648275]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

263

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x,y: x + y)

123433.11949172235

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(26, 334.1360125179572),
 (43, 339.6726151819133),
 (12, 328.0585177993378),
 (47, 343.4486408898434),
 (15, 318.91634844018785),
 (18, 332.8641649246866),
 (4, 334.12035783729465)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [184]:
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(50, 2777.696959541504),
 (20, 4214.479047385745),
 (40, 5449.469172051653),
 (30, 2158.938078822225),
 (10, 4391.378895688973),
 (11, 3113.512422503823),
 (31, 5537.261581317268),
 (21, 1619.9124917965592),
 (1, 1989.3379801015024),
 (41, 987.6845080551243)]

In [185]:
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(33, 8502.258699603124),
 (13, 8415.297489176775),
 (31, 5537.261581317268),
 (40, 5449.469172051653),
 (38, 4913.359912052394),
 (8, 4759.532719971837),
 (6, 4499.402344033714),
 (10, 4391.378895688973),
 (9, 4248.698412608107),
 (14, 4240.695095289299),
 (20, 4214.479047385745),
 (17, 4189.827591560459),
 (3, 3979.760150509989),
 (16, 3315.2678985587045),
 (29, 3167.526253511516),
 (27, 3128.799407596127),
 (11, 3113.512422503823),
 (22, 2982.062235528268),
 (46, 2923.9753596726296),
 (36, 2891.882573086974),
 (12, 2855.365305006478),
 (50, 2777.696959541504),
 (35, 2402.2629845885003),
 (48, 2378.743466938473),
 (15, 2287.2707284612197),
 (5, 2246.3162155189384),
 (30, 2158.938078822225),
 (49, 2122.302175933297),
 (1, 1989.3379801015024),
 (44, 1843.512625796834),
 (37, 1650.8160083774778),
 (21, 1619.9124917965592),
 (32, 1588.5481526564026),
 (47, 1395.6985306618362),
 (42, 1218.850852684454),
 (23, 1063.3081859076956),
 (41, 987.6845080551243),
 (34, 804.0661762797445),
 (28, 7

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [187]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(10, 16),
 (38, 13),
 (49, 13),
 (13, 12),
 (14, 12),
 (17, 10),
 (37, 10),
 (41, 9),
 (15, 9),
 (5, 8),
 (46, 8),
 (23, 7),
 (7, 7),
 (1, 6),
 (4, 6),
 (43, 6),
 (8, 6),
 (3, 6),
 (11, 6),
 (33, 5),
 (44, 5),
 (27, 5),
 (12, 5),
 (39, 5),
 (24, 5),
 (2, 5),
 (29, 5),
 (9, 4),
 (48, 4),
 (40, 4),
 (6, 4),
 (42, 4),
 (16, 4),
 (21, 4),
 (22, 3),
 (30, 3),
 (20, 3),
 (18, 3),
 (26, 3),
 (31, 3),
 (35, 2),
 (36, 2),
 (50, 2),
 (28, 2),
 (34, 2),
 (19, 2),
 (47, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
